In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
data={"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

with open('intents.json') as file:
    data = json.load(file)

In [3]:

with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

2/2 [==============================] - 0s 0s/step - loss: 2.0369 - accuracy: 0.3636
Epoch 74/500
2/2 [==============================] - 0s 0s/step - loss: 2.0359 - accuracy: 0.3636
Epoch 75/500
2/2 [==============================] - 0s 0s/step - loss: 2.0351 - accuracy: 0.3636
Epoch 76/500
2/2 [==============================] - 0s 0s/step - loss: 2.0340 - accuracy: 0.3636
Epoch 77/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0330 - accuracy: 0.3636
Epoch 78/500
2/2 [==============================] - 0s 18ms/step - loss: 2.0318 - accuracy: 0.3636
Epoch 79/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0306 - accuracy: 0.3636
Epoch 80/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0293 - accuracy: 0.3636
Epoch 81/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0278 - accuracy: 0.3636
Epoch 82/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0262 - accuracy: 0.3636
Epoch 83/500
2/2 [==============

2/2 [==============================] - 0s 7ms/step - loss: 1.8308 - accuracy: 0.3636
Epoch 157/500
2/2 [==============================] - 0s 9ms/step - loss: 1.8266 - accuracy: 0.3636
Epoch 158/500
2/2 [==============================] - 0s 16ms/step - loss: 1.8229 - accuracy: 0.3636
Epoch 159/500
2/2 [==============================] - 0s 16ms/step - loss: 1.8201 - accuracy: 0.3636
Epoch 160/500
2/2 [==============================] - 0s 15ms/step - loss: 1.8181 - accuracy: 0.3636
Epoch 161/500
2/2 [==============================] - 0s 13ms/step - loss: 1.8146 - accuracy: 0.3636
Epoch 162/500
2/2 [==============================] - 0s 0s/step - loss: 1.8103 - accuracy: 0.3636
Epoch 163/500
2/2 [==============================] - 0s 0s/step - loss: 1.8060 - accuracy: 0.3636
Epoch 164/500
2/2 [==============================] - 0s 11ms/step - loss: 1.8018 - accuracy: 0.3636
Epoch 165/500
2/2 [==============================] - 0s 10ms/step - loss: 1.7981 - accuracy: 0.3636
Epoch 166/500
2/2 [=

2/2 [==============================] - 0s 0s/step - loss: 1.4568 - accuracy: 0.3636
Epoch 239/500
2/2 [==============================] - 0s 9ms/step - loss: 1.4530 - accuracy: 0.3636
Epoch 240/500
2/2 [==============================] - 0s 5ms/step - loss: 1.4478 - accuracy: 0.3636
Epoch 241/500
2/2 [==============================] - 0s 7ms/step - loss: 1.4424 - accuracy: 0.3636
Epoch 242/500
2/2 [==============================] - 0s 7ms/step - loss: 1.4393 - accuracy: 0.3636
Epoch 243/500
2/2 [==============================] - 0s 0s/step - loss: 1.4375 - accuracy: 0.3636
Epoch 244/500
2/2 [==============================] - 0s 9ms/step - loss: 1.4362 - accuracy: 0.3636
Epoch 245/500
2/2 [==============================] - 0s 0s/step - loss: 1.4343 - accuracy: 0.3636
Epoch 246/500
2/2 [==============================] - 0s 0s/step - loss: 1.4309 - accuracy: 0.3636
Epoch 247/500
2/2 [==============================] - 0s 0s/step - loss: 1.4245 - accuracy: 0.3636
Epoch 248/500
2/2 [==========

2/2 [==============================] - 0s 9ms/step - loss: 1.1587 - accuracy: 0.5152
Epoch 321/500
2/2 [==============================] - 0s 0s/step - loss: 1.1551 - accuracy: 0.5152
Epoch 322/500
2/2 [==============================] - 0s 0s/step - loss: 1.1524 - accuracy: 0.5455
Epoch 323/500
2/2 [==============================] - 0s 0s/step - loss: 1.1497 - accuracy: 0.5455
Epoch 324/500
2/2 [==============================] - 0s 0s/step - loss: 1.1449 - accuracy: 0.5455
Epoch 325/500
2/2 [==============================] - 0s 0s/step - loss: 1.1401 - accuracy: 0.5152
Epoch 326/500
2/2 [==============================] - 0s 0s/step - loss: 1.1371 - accuracy: 0.6364
Epoch 327/500
2/2 [==============================] - 0s 0s/step - loss: 1.1349 - accuracy: 0.6667
Epoch 328/500
2/2 [==============================] - 0s 0s/step - loss: 1.1328 - accuracy: 0.6667
Epoch 329/500
2/2 [==============================] - 0s 441us/step - loss: 1.1299 - accuracy: 0.6667
Epoch 330/500
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 0.8439 - accuracy: 0.7879
Epoch 403/500
2/2 [==============================] - 0s 18ms/step - loss: 0.8388 - accuracy: 0.7879
Epoch 404/500
2/2 [==============================] - 0s 4ms/step - loss: 0.8356 - accuracy: 0.7879
Epoch 405/500
2/2 [==============================] - 0s 15ms/step - loss: 0.8329 - accuracy: 0.7879
Epoch 406/500
2/2 [==============================] - 0s 7ms/step - loss: 0.8306 - accuracy: 0.7879
Epoch 407/500
2/2 [==============================] - 0s 8ms/step - loss: 0.8282 - accuracy: 0.7879
Epoch 408/500
2/2 [==============================] - 0s 15ms/step - loss: 0.8218 - accuracy: 0.8485
Epoch 409/500
2/2 [==============================] - 0s 14ms/step - loss: 0.8124 - accuracy: 0.7879
Epoch 410/500
2/2 [==============================] - 0s 5ms/step - loss: 0.8031 - accuracy: 0.7879
Epoch 411/500
2/2 [==============================] - 0s 11ms/step - loss: 0.7988 - accuracy: 0.7879
Epoch 412/500
2/2 [

2/2 [==============================] - 0s 0s/step - loss: 0.5517 - accuracy: 0.9697
Epoch 485/500
2/2 [==============================] - 0s 17ms/step - loss: 0.5496 - accuracy: 0.9394
Epoch 486/500
2/2 [==============================] - 0s 10ms/step - loss: 0.5471 - accuracy: 0.9394
Epoch 487/500
2/2 [==============================] - 0s 6ms/step - loss: 0.5442 - accuracy: 0.9091
Epoch 488/500
2/2 [==============================] - 0s 17ms/step - loss: 0.5411 - accuracy: 0.9091
Epoch 489/500
2/2 [==============================] - 0s 0s/step - loss: 0.5386 - accuracy: 0.9091
Epoch 490/500
2/2 [==============================] - 0s 0s/step - loss: 0.5368 - accuracy: 0.9091
Epoch 491/500
2/2 [==============================] - 0s 15ms/step - loss: 0.5367 - accuracy: 0.9394
Epoch 492/500
2/2 [==============================] - 0s 17ms/step - loss: 0.5382 - accuracy: 0.8788
Epoch 493/500
2/2 [==============================] - 0s 10ms/step - loss: 0.5397 - accuracy: 0.8788
Epoch 494/500
2/2 [==

In [8]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [9]:
from tensorflow.keras.models import load_model

# Load the trained model
loaded_model = load_model("chat_model")


In [10]:
import pickle

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

# Load the label encoder
with open('label_encoder.pickle', 'rb') as ecn_file:
    loaded_label_encoder = pickle.load(ecn_file)


In [11]:
def get_user_input():
    user_input = input("You: ")
    return user_input


In [12]:
def preprocess_input(user_input, tokenizer, max_len):
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)
    return padded_sequence


In [13]:
def get_model_response(model, user_input, tokenizer, max_len):
    processed_input = preprocess_input(user_input, tokenizer, max_len)
    model_output = model.predict(processed_input)
    return model_output


In [14]:
def postprocess_output(model_output, label_encoder):
    predicted_label = np.argmax(model_output, axis=1)
    response = label_encoder.inverse_transform(predicted_label)
    return response


In [15]:
while True:
    user_input = get_user_input()
    if user_input.lower() == 'exit':
        break

    model_output = get_model_response(loaded_model, user_input, loaded_tokenizer, max_len)
    response = postprocess_output(model_output, loaded_label_encoder)

    print(f"Bot: {response[3]}")


You: Who are you
1/1 [==============================] - 0s 169ms/step


IndexError: index 3 is out of bounds for axis 0 with size 1